# Imports and Pre-requisites

In [39]:
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime

In [40]:
ee.Authenticate()

True

In [41]:
ee.Initialize(project="rwanda-climate-alerts")

# Collect Datasets

## Baseline setup
In this section, we will define the
- baseline area of interest: polygon covering Rwanda
- drought/flood years

In [42]:
# drought_years = [
#     "1984-10-01",
#     "1989-12-01",
#     "1996-01-01",
#     "1999-11-01",
#     "2000-01-01",  # "Early 2000"
#     "2003-03-01",
#     "2005-02-01",
#     "2006-03-01",
#     "2006-09-01",
#     "2014-06-01"
# ]
#
# # Flood dates
# flood_years = [
#     "1988-05-06",
#     "2000-11-21",
#     "2001-09-22",
#     "2001-10-30",
#     "2002-04-26",
#     "2003-10-30",
#     "2005-08-16"
# ]

In [43]:
# Convert date string to date format
def str_to_date_range(date_string, date_format):
    initial_date = datetime.strptime(date_string, date_format)

    two_weeks_early_date = initial_date - timedelta(weeks=2)
    two_weeks_later_date = initial_date + timedelta(weeks=2)

    return two_weeks_early_date, two_weeks_later_date

# date_1, date_2 = str_to_date_range("1970-01-01", "%Y-%m-%d")
# print(date_1, date_2)

def date_range_to_list(date_list):
    date_range_list = []

    for flood_date in date_list:
        date_range_0, date_range_1 = str_to_date_range(flood_date, "%Y-%m-%d")
        date_range_list.append([date_range_0, date_range_1])

    return date_range_list

flood_dates_range = []

for flood_range in flood_dates_range:
    print(flood_range[0], flood_range[1])

In [44]:
aoi = ee.Geometry.Polygon([
    [
        [28.70, -2.85],
        [31.00, -2.85],
        [31.00, -1.00],
        [28.70, -1.00]
    ]
])

# Drought dates
drought_years = ["1984", "1989", "1996", "1999", "2000", "2003", "2005", "2006", "2006", "2014"]

# Flood dates
flood_years = ["1988", "2000", "2001", "2001", "2002", "2003", "2005"]

## Fetch Datasets

In [65]:
# Create map
Map = geemap.Map()

districts = ee.FeatureCollection("FAO/GAUL/2015/level2") \
                .filter(ee.Filter.eq("ADM0_NAME", "Rwanda")) \
                .map(lambda feature: feature.geometry())

Map.addLayer(districts, {}, "Rwanda Districts")
Map.centerObject(districts, 7)

In [66]:
# CHIRPS Daily Rainfall (mm/day)
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
            .filterDate("2001-01-01", "2025-08-01")\
            .filterBounds(districts) \
            .map(lambda img: img.clip(districts))

# ERA5 Land Monthly Temperature
era5_temp = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
                .select("temperature_2m") \
                .filterDate("2001-01-01", "2025-08-01") \
                .filterBounds(districts) \
                .map(lambda img: img.clip(districts))

# Soil Moisture (ERA5 Land)
soil_moist = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
                .select("volumetric_soil_water_layer_1") \
                .filterDate("2001-01-01", "2025-08-01") \
                .filterBounds(districts) \
                .map(lambda img: img.clip(districts))

# MODIS NDVI (monthly, 250 m)
ndvi = ee.ImageCollection("MODIS/006/MOD13A1") \
            .select("NDVI") \
            .filterDate("2001-01-01", "2025-08-01") \
            .filterBounds(districts) \
            .map(lambda img: img.clip(districts))

# ESA WorldCover 2020 (10 m)
landcover = ee.Image("ESA/WorldCover/v200/2022").clip(districts)

# DEM (SRTM ~30m resolution)
dem = ee.Image("USGS/SRTMGL1_003").clip(districts)

# Calculate slope (degrees)
slope = ee.Terrain.slope(dem)


# MODIS Land Cover (optional, for time series)
modis_lc = ee.ImageCollection("MODIS/006/MCD12Q1") \
                .select("LC_Type1") \
                .filterDate("2001-01-01", "2020-12-31") \
                .filterBounds(districts) \
                .map(lambda img: img.clip(districts))

## Aggregate monthly

In [67]:
def aggregate_monthly(image_collection):
    monthly_result = image_collection.map(lambda img: img.set("month", img.date().format("YYYY-MM")))
    monthly_sum = monthly_result.reduce(ee.Reducer.sum())
    return monthly_result, monthly_sum

monthly_chirps, monthly_chirps_sum = aggregate_monthly(chirps)
monthly_era5_temp, monthly_era5_temp_sum = aggregate_monthly(era5_temp)
monthly_soil_moist, monthly_soil_moist_sum = aggregate_monthly(soil_moist)
monthly_ndvi, monthly_ndvi_sum = aggregate_monthly(ndvi)

## Calculate historical baseline and anomalies

In [68]:
def calculate_baseline(image_collection):
    # Define baseline period (e.g., 2000–2015)
    baseline = image_collection.filterDate("2001-01-01", "2015-12-31")

    # Mean rainfall baseline
    baseline_mean = baseline.mean()

    # Recent period (e.g., 2020–2025)
    recent = image_collection.filterDate("2025-07-01", "2025-08-01").mean()

    # Rainfall anomaly (recent vs baseline)
    anomaly = recent.subtract(baseline_mean).divide(baseline_mean)

    return baseline, baseline_mean, anomaly

rain_baseline, rain_baseline_mean, rain_anomaly = calculate_baseline(chirps)
temp_baseline, temp_baseline_mean, temp_anomaly = calculate_baseline(era5_temp)
soil_moist_baseline, soil_moist_baseline_mean, soil_moist_anomaly = calculate_baseline(soil_moist)
ndvi_baseline, ndvi_baseline_mean, ndvi_anomaly = calculate_baseline(ndvi)

## Normalize indices (0-1)

In [69]:
def normalize(img, region):
    min_value = img.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=region,
        scale=1000,
        maxPixels=1e13
    )
    max_value = img.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=region,
        scale=1000,
        maxPixels=1e13
    )

    return img.subtract(min_value).divide(ee.Number(max_value).subtract(min_value))

rain_norm = normalize(rain_anomaly, districts)
temp_norm = normalize(temp_anomaly, districts)
soil_moist_norm = normalize(soil_moist_anomaly, districts)
ndvi_norm = normalize(ndvi_anomaly, districts)

dem_norm = normalize(dem, districts)
slope_norm = normalize(slope, districts)

## Cobine risk indexes

In [78]:
flood_risk_index = rain_norm.multiply(0.5) \
                    .add(temp_norm.multiply(0.25)) \
                    .add(ndvi_norm.multiply(0.25)) \
                    .add(slope_norm.multiply(0.2))

drought_risk_index = rain_norm.multiply(0.4) \
                    .add(soil_moist_norm.multiply(0.4)) \
                    .add(ndvi_norm.multiply(0.2))

landslide_risk_index = rain_norm.multiply(0.4) \
                        .add(soil_moist_norm.multiply(0.4)) \
                        .add(slope_norm.multiply(0.2))

## Aggregate risk by district

In [73]:
def aggregate_risk(risk_index):
    district_stats = risk_index.reduceRegions(
        collection=districts,
        reducer=ee.Reducer.mean(),
        scale=1000
    )
    return district_stats

flood_risk_stats = aggregate_risk(flood_risk_index)
drought_risk_stats = aggregate_risk(drought_risk_index)
landslide_risk_stats = aggregate_risk(landslide_risk_index)

# Print first feature
print(flood_risk_stats.first().getInfo())

EEException: Image.subtract, argument 'image2': Invalid type.
Expected type: Image<unknown bands>.
Actual type: Dictionary<Float>.
Actual value: {precipitation=-0.9888367602330774}

## Compute pixel-level hazard indices

In [84]:
# Example: Monthly rainfall anomaly
monthly_rain = chirps_h\
    .map(lambda img: img.set("month", img.date().get("month")))

baseline_mean = monthly_rain.mean()

In [70]:
# m = geemap.Map(center=[-1.94, 29.87], zoom=8)
#
# m.addLayer(aoi, {'color': 'black'}, 'Geometry [black]: Rwanda')
#
# m.addLayer(annual_rain_aoi, vis_params, 'CHIRPS')
# palette = ['red', 'orange', 'yellow', 'green', 'blue', 'darkblue']
# vis_params = {
#     'min': 0,
#     'max': 200000, # Adjust max based on expected precipitation range (in mm)
#     'palette': palette
# }
#
# m.add_colorbar(vis_params, label="Rainfall (mm)")
#
# m

Map(center=[-1.94, 29.87], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…